In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("db_measurements_v2.1.0.csv")

<ipython-input-17-41ce8bf8db1f>:1: DtypeWarning: Columns (5,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("db_measurements_v2.1.0.csv")


In [ ]:
df.columns

Index(['index', 'record_id', 'building_id', 'timestamp', 'season',
       'subject_id', 'age', 'gender', 'ht', 'wt', 'ta', 'ta_h', 'ta_m', 'ta_l',
       'top', 'tr', 'tg', 'tg_h', 'tg_m', 'tg_l', 'rh', 'vel', 'vel_h',
       'vel_m', 'vel_l', 'met', 'clo', 'activity_10', 'activity_20',
       'activity_30', 'activity_60', 'thermal_sensation',
       'thermal_acceptability', 'thermal_preference', 'thermal_comfort',
       'air_movement_acceptability', 'air_movement_preference',
       'blind_curtain', 'fan', 'window', 'door', 'heater', 't_out', 'rh_out',
       't_out_isd', 'rh_out_isd', 'set', 'pmv', 'ppd', 'pmv_ce', 'ppd_ce',
       't_mot_isd'],
      dtype='object')

In [ ]:
considered = df[["ta","tr","vel","rh","met","clo"]]
predictors = df[['pmv', 'ppd', 'pmv_ce', 'ppd_ce']]
classifiers = df[["thermal_acceptability","thermal_preference","thermal_comfort"]]

In [ ]:
df[["thermal_acceptability","thermal_preference","thermal_comfort"]]

,thermal_acceptability,thermal_preference,thermal_comfort
0,acceptable,cooler,4.0
1,unacceptable,cooler,2.0
2,acceptable,no change,5.0
3,acceptable,no change,5.0
4,acceptable,no change,5.0
...,...,...,...
109028,NaN,NaN,NaN
109029,NaN,NaN,NaN
109030,NaN,NaN,NaN
109031,NaN,NaN,NaN


In [ ]:
df[['pmv', 'ppd', 'pmv_ce', 'ppd_ce']]

,pmv,ppd,pmv_ce,ppd_ce
0,0.50,10.2,0.38,8.0
1,0.40,8.4,0.40,8.4
2,-0.07,5.1,-0.07,5.1
3,0.31,7.0,0.14,5.4
4,0.05,5.0,-0.06,5.1
...,...,...,...,...
109028,NaN,NaN,NaN,NaN
109029,NaN,NaN,NaN,NaN
109030,NaN,NaN,NaN,NaN
109031,NaN,NaN,NaN,NaN


In [ ]:
new_df = df[["ta","tr","vel","rh","met","clo",'pmv', 'ppd', 'pmv_ce', 'ppd_ce',"thermal_acceptability","thermal_preference","thermal_comfort"]]

In [ ]:
new_df = new_df.dropna(axis=0)

In [ ]:
new_df.columns

Index(['ta', 'tr', 'vel', 'rh', 'met', 'clo', 'pmv', 'ppd', 'pmv_ce', 'ppd_ce',
       'thermal_acceptability', 'thermal_preference', 'thermal_comfort'],
      dtype='object')

In [ ]:
new_df.dtypes

ta                       float64
tr                       float64
vel                      float64
rh                       float64
met                      float64
clo                      float64
pmv                      float64
ppd                      float64
pmv_ce                   float64
ppd_ce                   float64
thermal_acceptability     object
thermal_preference        object
thermal_comfort          float64
dtype: object

In [ ]:
new_df["thermal_acceptability"].value_counts()

acceptable      5582
unacceptable     800
Name: thermal_acceptability, dtype: int64

In [ ]:
new_df["thermal_acceptability"] = new_df["thermal_acceptability"].apply(lambda row : 1 if row=="acceptable" else 0)

In [ ]:
new_df["thermal_preference"].value_counts()

no change    3698
cooler       1581
warmer       1103
Name: thermal_preference, dtype: int64

In [ ]:
new_df["thermal_comfort"].value_counts()

5.0    3193
4.0    1245
6.0     909
3.0     757
2.0     205
1.0      73
Name: thermal_comfort, dtype: int64

In [ ]:
X = new_df[["ta","tr","vel","rh","met","clo"]]
y = new_df[["pmv"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
mean_squared_error(y_pred,y_test)
r2_score(y_pred,y_test)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9938482448821345

In [ ]:
# save the model to disk
import pickle
filename = 'pmv_model.sav'
pickle.dump(regressor, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))
result = regressor.predict(X_test)
print(r2_score(result,y_test))

0.9938482448821345


In [ ]:
X1 = new_df[["ta","tr","vel","rh","met","clo"]]
y1 = new_df[["ppd"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X1 = sc_X.fit_transform(X1)
y1 = sc_y.fit_transform(y1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1,random_state=1)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train1, y_train1)
y_pred1 = regressor.predict(X_test1)
mean_squared_error(y_pred1,y_test1)
r2_score(y_pred1,y_test1)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9746474966190946

In [ ]:
# save the model to disk
import pickle
filename = 'ppd_model.sav'
pickle.dump(regressor, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))
result = regressor.predict(X_test1)
print(r2_score(result,y_test1))

0.9746474966190946


# **UNSUPERVISED LEARNING FOR THESE ATTRIBUTES**

In [ ]:
X2 = new_df[["ta","tr","vel","rh","met","clo","pmv","ppd"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df_kmeans = scaler.fit_transform(X2)

In [ ]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)
clusters = kmeans_model.fit_predict(scaled_df_kmeans)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# save the model to disk
import pickle
filename = 'unsup_kMeans_model.sav'
pickle.dump(kmeans_model, open(filename, 'wb'))

In [ ]:
pd.DataFrame(clusters).value_counts()

0    3972
1    1279
2    1131
dtype: int64

In [ ]:
X2

,ta,tr,vel,rh,met,clo,pmv,ppd
0,22.3,22.950000,0.03,61.0,1.706485,0.95,0.50,10.2
1,23.0,24.134286,0.08,59.0,1.109215,1.07,0.40,8.4
2,22.0,22.091429,0.04,61.0,1.211604,0.88,-0.07,5.1
3,23.2,24.330303,0.03,58.0,1.774744,0.59,0.31,7.0
4,22.2,23.077143,0.15,62.0,1.484642,0.83,0.05,5.0
...,...,...,...,...,...,...,...,...
96297,24.5,24.700000,0.03,72.0,1.200000,0.51,0.17,5.6
96298,24.5,24.700000,0.03,72.0,1.200000,0.58,0.29,6.8
96299,24.5,24.700000,0.03,72.0,1.200000,0.51,0.17,5.6
96300,24.5,24.700000,0.03,72.0,1.200000,0.67,0.44,9.0


In [ ]:
new_df.iloc[0]

ta                           22.3
tr                          22.95
vel                          0.03
rh                           61.0
met                      1.706485
clo                          0.95
pmv                           0.5
ppd                          10.2
pmv_ce                       0.38
ppd_ce                        8.0
thermal_acceptability           1
thermal_preference         cooler
thermal_comfort               4.0
Name: 0, dtype: object

In [103]:
cluster0 = []
cluster1 = []
cluster2 = []
cluster0 = {
            'ta': [0],
            'tr':[0],
            'vel':[0],
            'rh':[0], 
            'met':[0], 
            'clo':[0],
            'pmv':[0], 
            'ppd':[0], 
            'pmv_ce':[0],
            'ppd_ce':[0], 
            'thermal_acceptability':[0],
            'thermal_preference':[0], 
            'thermal_comfort':[0], 
}
cluster0 = pd.DataFrame(cluster0)


cluster1 = {
            'ta': [0],
            'tr':[0],
            'vel':[0],
            'rh':[0], 
            'met':[0], 
            'clo':[0],
            'pmv':[0], 
            'ppd':[0], 
            'pmv_ce':[0],
            'ppd_ce':[0], 
            'thermal_acceptability':[0],
            'thermal_preference':[0], 
            'thermal_comfort':[0], 
}
cluster1 = pd.DataFrame(cluster1)


cluster2 = {
            'ta': [0],
            'tr':[0],
            'vel':[0],
            'rh':[0], 
            'met':[0], 
            'clo':[0],
            'pmv':[0], 
            'ppd':[0], 
            'pmv_ce':[0],
            'ppd_ce':[0], 
            'thermal_acceptability':[0],
            'thermal_preference':[0], 
            'thermal_comfort':[0], 
}
cluster2 = pd.DataFrame(cluster2)



for i in range(0,len(new_df)):
  if(clusters[i]==0):
    cluster0 = cluster0.append(new_df.iloc[i])
  elif(clusters[i]==1):
    cluster1 = cluster1.append(new_df.iloc[i])
  else:
    cluster2 = cluster2.append(new_df.iloc[i])

Streaming output truncated to the last 5000 lines.
<ipython-input-103-ddbf279392c4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster1 = cluster1.append(new_df.iloc[i])
<ipython-input-103-ddbf279392c4>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster0 = cluster0.append(new_df.iloc[i])
<ipython-input-103-ddbf279392c4>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster0 = cluster0.append(new_df.iloc[i])
<ipython-input-103-ddbf279392c4>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster0 = cluster0.append(new_df.iloc[i])
<ipython-input-103-ddbf279392c4>:65: FutureWarning: The frame.append method is deprecated

In [104]:
cluster0.to_csv("one.csv")
cluster1.to_csv("two.csv")
cluster2.to_csv("three.csv")

In [105]:
df

,index,record_id,building_id,timestamp,season,subject_id,age,gender,ht,wt,...,t_out,rh_out,t_out_isd,rh_out_isd,set,pmv,ppd,pmv_ce,ppd_ce,t_mot_isd
0,0,1,1,1995-05-18T00:00:00Z,winter,10.0,NaN,female,NaN,NaN,...,15.86,99.1,16.295833,86.837514,28.1,0.50,10.2,0.38,8.0,17.4
1,1,2,1,1995-05-18T00:00:00Z,winter,14.0,NaN,female,NaN,NaN,...,15.86,99.1,16.295833,86.837514,27.2,0.40,8.4,0.40,8.4,17.4
2,2,3,1,1995-05-18T00:00:00Z,winter,15.0,NaN,female,NaN,NaN,...,15.86,99.1,16.295833,86.837514,24.8,-0.07,5.1,-0.07,5.1,17.4
3,3,4,1,1995-05-18T00:00:00Z,winter,17.0,NaN,female,NaN,NaN,...,15.86,99.1,16.295833,86.837514,25.5,0.31,7.0,0.14,5.4,17.4
4,4,5,1,1995-05-18T00:00:00Z,winter,21.0,NaN,male,NaN,NaN,...,15.86,99.1,16.295833,86.837514,26.0,0.05,5.0,-0.06,5.1,17.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109028,110065,110066,809,2014-06-02T00:00:00Z,hot/wet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.316667,82.059739,NaN,NaN,NaN,NaN,NaN,28.0
109029,110066,110067,809,2014-06-02T00:00:00Z,hot/wet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.316667,82.059739,NaN,NaN,NaN,NaN,NaN,28.0
109030,110067,110068,809,2014-06-02T00:00:00Z,hot/wet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.316667,82.059739,NaN,NaN,NaN,NaN,NaN,28.0
109031,110068,110069,809,2014-06-02T00:00:00Z,hot/wet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.316667,82.059739,NaN,NaN,NaN,NaN,NaN,28.0


In [107]:
second = pd.read_csv("two.csv")

In [109]:
second.dtypes

Unnamed: 0                 int64
ta                       float64
tr                       float64
vel                      float64
rh                       float64
met                      float64
clo                      float64
pmv                      float64
ppd                      float64
pmv_ce                   float64
ppd_ce                   float64
thermal_acceptability      int64
thermal_preference        object
thermal_comfort          float64
dtype: object

In [111]:
second["thermal_comfort"].value_counts()

5.0    672
6.0    214
4.0    198
3.0    142
2.0     41
1.0     12
0.0      1
Name: thermal_comfort, dtype: int64

In [112]:
second.corr()

,Unnamed: 0,ta,tr,vel,rh,met,clo,pmv,ppd,pmv_ce,ppd_ce,thermal_acceptability,thermal_comfort
Unnamed: 0,1.000000,-0.269818,-0.172875,0.139525,0.409950,0.049248,0.214011,-0.047778,0.064101,-0.075340,0.089336,0.097630,0.084045
ta,-0.269818,1.000000,0.962695,0.453734,-0.246542,-0.325632,-0.816464,-0.066888,0.010235,-0.011413,-0.044955,0.022439,0.024569
tr,-0.172875,0.962695,1.000000,0.539354,-0.176448,-0.293345,-0.809658,-0.084426,0.028980,-0.030086,-0.025906,0.038173,0.049968
vel,0.139525,0.453734,0.539354,1.000000,0.029718,-0.050460,-0.394019,-0.238433,0.215041,-0.229320,0.203423,0.023676,0.072294
rh,0.409950,-0.246542,-0.176448,0.029718,1.000000,0.211970,0.110780,0.006945,0.005556,-0.000841,0.012536,0.054239,0.140260
met,0.049248,-0.325632,-0.293345,-0.050460,0.211970,1.000000,0.122179,0.203770,-0.194821,0.147936,-0.144268,-0.015629,0.082309
clo,0.214011,-0.816464,-0.809658,-0.394019,0.110780,0.122179,1.000000,0.367148,-0.311469,0.320756,-0.263682,-0.018818,-0.043180
pmv,-0.047778,-0.066888,-0.084426,-0.238433,0.006945,0.203770,0.367148,1.000000,-0.990692,0.985359,-0.974277,-0.053582,-0.061568
ppd,0.064101,0.010235,0.028980,0.215041,0.005556,-0.194821,-0.311469,-0.990692,1.000000,-0.978955,0.986184,0.054875,0.058826
pmv_ce,-0.075340,-0.011413,-0.030086,-0.229320,-0.000841,0.147936,0.320756,0.985359,-0.978955,1.000000,-0.990417,-0.049604,-0.056435


In [123]:
second_df = second.drop(0,axis=0)
second_df = second.drop("Unnamed: 0",axis=1)

In [130]:
X = second_df.iloc[:,:10]
y = second_df["thermal_acceptability"]

In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# create a logistic regression model
lr = LogisticRegression()

# fit the model with the training data
lr.fit(X_train, y_train)

# make predictions on the testing data
y_pred = lr.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8411458333333334


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# create a Random Forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100)

# fit the model with the training data
rf.fit(X_train, y_train)

# make predictions on the testing data
y_pred = rf.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8333333333333334


In [133]:
# save the model to disk
import pickle
filename = 'thermal_acc_model.sav'
pickle.dump(rf, open(filename, 'wb'))


In [134]:
X1 = second_df.iloc[:,:10]
y1 = second_df["thermal_comfort"]

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

# create a logistic regression model
lr1 = LogisticRegression()

# fit the model with the training data
lr1.fit(X_train, y_train)

# make predictions on the testing data
y_pred = lr1.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [137]:
# save the model to disk
import pickle
filename = 'thermal_comfort_model.sav'
pickle.dump(lr1, open(filename, 'wb'))
